In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
seed = 1
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [8]:
class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BiLSTM, self).__init__()
        self.hidden_size = hidden_size

        # Defining the first bidirectional LSTM layer
        self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True)

        # Defining the second bidirectional LSTM layer
        self.lstm2 = nn.LSTM(2 * hidden_size, hidden_size, batch_first=True, bidirectional=True)

        # Fully connected layer that outputs the predicted value
        self.fc = nn.Linear(2 * hidden_size, output_size)

    def forward(self, x):
        # Initial hidden and cell states are zero
        batch_size = x.size(0)
        h0 = torch.zeros(2, batch_size, self.hidden_size).to(x.device)  # 2 for bidirectional
        c0 = torch.zeros(2, batch_size, self.hidden_size).to(x.device)  # 2 for bidirectional

        # First bidirectional LSTM layer
        out, (hn, cn) = self.lstm1(x, (h0, c0))

        # Second bidirectional LSTM layer
        out, (hn, cn) = self.lstm2(out, (hn, cn))

        # Reshape the output from the second LSTM layer to fit the fully connected layer
        out = out[:, -1, :]  # Taking the last time step's output

        # Fully connected layer
        out = self.fc(out)
        return out

In [9]:
# Parameters
input_size = 7
hidden_size = 128
output_size = 1

# Create the model
model = BiLSTM(input_size, hidden_size, output_size)
print(summary(model))
torch.save(model,'modelCache/BiLSTM.pth')

Layer (type:depth-idx)                   Param #
BiLSTM                                   --
├─LSTM: 1-1                              140,288
├─LSTM: 1-2                              395,264
├─Linear: 1-3                            257
Total params: 535,809
Trainable params: 535,809
Non-trainable params: 0


In [10]:
seq_len = 1  # Sequence length
batch_size = 1 # Number of sequences in the batch
x = torch.randn(batch_size, seq_len, input_size)

model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    output = model(x)

print("Input shape:", x.shape)
print("Output shape:", output.shape)
print("Output:", output)

Input shape: torch.Size([1, 1, 7])
Output shape: torch.Size([1, 1])
Output: tensor([[0.0256]])
